In [1]:
%run classifier.py
from elasticsearch import Elasticsearch
from elasticsearch import helpers
scan = helpers.scan

loading E
loading S
loading T
loading J


In [2]:
es = Elasticsearch([{'host':'localhost','port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [3]:
def classify(es, author, idx):
    classifier = None
    src_results = scan(es, scroll='10m',    
        query = {
            "query": {
                "query_string" : {
                    "query" : author,
                    "default_field" : "author"
                }
            }
        }, index = idx)
    
    count=0
    for src_doc in src_results:
        count = count + 1
        if count >= 100:
            break # limit to 100 comments for performance
        
        if classifier is None:
            classifier = Classifier() 
        text = src_doc['_source']['text_body']
        classifier.preprocess(text, web=False)
        
    return classifier

In [5]:
import random
src_index = 'redditors.followers.sample'
dest_index = 'redditors.followers.sample.comments'

src_results = es.search(scroll='360m',
    body = {
    "query": {
      "function_score": {
        "query": {
          "bool": {
            "must": [
              {
                "bool": {
                  "must_not": [
                    {
                      "exists": {
                        "field": "bm"
                      }
                    }
                  ]
                }
              },
              {
                "term": {
                  "comments_indexed": {
                    "value": True
                  }
                }
              }
            ]
          }
        },
        "random_score": {
            "seed": random.randint(1,1000)
        },
        "boost_mode": "replace"
      }
    }
  }, index = src_index, size=1000)

for src_doc in src_results['hits']['hits']:
    author_doc = src_doc['_source']
    print(author_doc['name'])
    update = None
    
    classifier = classify(es, author_doc['name'], dest_index)
    if classifier is not None:
        if update is None:
            update = { "doc": { "bm": {} } }

        res = more_magic(classifier)
        res['type'] = ''.join([key[0] for key in res.keys()])
        update['doc']['bm']['overall'] = res

    if update is not None:
        es.update(index = src_index, id = src_doc['_id'], body = update)
    else:
        update = { "doc": { "bm": {} } }
        es.update(index = src_index, id = src_doc['_id'], body = update)

blosserraptor
redditors.followers.sample.comments
sulcatatortoise7
redditors.followers.sample.comments
slickwilliefitz
redditors.followers.sample.comments
UndeadWorm
redditors.followers.sample.comments
anonyngineer
redditors.followers.sample.comments
kimtaekkat
redditors.followers.sample.comments
Intothefireandice
redditors.followers.sample.comments
K-Zoro
redditors.followers.sample.comments


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jason/.local/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'POS_RB_std'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jason/.local/lib/python3.7/site-packages/pandas/core/internals/managers.py", line 1069, in set
    loc = self.items.get_loc(item)
  File "/home/jason/.local/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2899, in get_loc
    return self._engine.get_loc(self._maybe_

TypeError: can only concatenate str (not "list") to str